<a href="https://colab.research.google.com/github/Komsomolochka/sentiment_analysis/blob/main/sentiment_analysis_week1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [93]:
from nltk.corpus import movie_reviews
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import numpy as np
import pandas as pd

In [5]:
import nltk
nltk.download('movie_reviews')


[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.


True

In [49]:
def write_answer(file_name, *answer):
    print(*answer, file=open(file_name, "w"))

In [6]:
#id позитивных и негативных отзывов
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

In [41]:
#списки позитивных и негативных отзывов
negfeats = [" ".join(movie_reviews.words(fileids=[f])) for f in negids]
posfeats = [" ".join(movie_reviews.words(fileids=[f])) for f in posids]

1. Создайте список из текстов всех имеющихся отзывов, а также список с классами, которые будет использовать ваш классификатор - 0 для негативных отзывов и 1 для позитивных.

In [27]:
target = len(negids) * [0] + len(posids) * [1] 

In [42]:
text = negfeats + posfeats

2. Подсчитайте количество отзывов в выборке.

In [43]:
len(text)

2000

In [50]:
write_answer('q1.txt', len(text))

3. Подсчитайте долю класса 1 в выборке.

In [35]:
len(negids)/len(target)

0.5

In [51]:
write_answer('q2.txt', len(negids)/len(target))

4. Импортируйте CountVectorizer из sklearn.feature_extraction.text. Попробуйте использовать его с настройками по умолчанию для того, чтобы получить признаковое представление каждого текста. Скорее всего, попытка не увенчается успехом. Разберитесь, в чем причина, и добейтесь того, чтобы метод fit_transform у CountVectorizer успешно отрабатывал. Подсчитайте количество признаков в CountVectorizer. Никакой предварительной обработки текста (удаление стоп-слов, нормализация слов) на этом шаге делать не надо, в качестве признаков должны использоваться частоты слов.



In [47]:
vectorizer = CountVectorizer()
vect_text = vectorizer.fit_transform(text)

In [48]:
vect_text.shape

(2000, 39659)

In [52]:
write_answer('q3.txt', vect_text.shape[1])

5. Соберите pipeline из CountVectorizer и LogisticRegression c настройками по-умолчанию и с помощью cross_val_score (также со стандартными настройками) оцените получаемое "из коробки" качество по accuracy.

In [70]:
pipeline = Pipeline([('vectorizer', CountVectorizer()),('classifier', LogisticRegression(max_iter=1000))])

In [71]:
text_cv = cross_val_score(pipeline, text, target, scoring="accuracy")

In [72]:
text_cv

array([0.8   , 0.8475, 0.845 , 0.875 , 0.855 ])

In [73]:
write_answer('q4.txt', text_cv.mean())

6. Аналогично accuracy, оцените качество по ROC AUC.

In [74]:
text_cv_roc = cross_val_score(pipeline, text, target, scoring="roc_auc")

In [75]:
text_cv_roc

array([0.896975, 0.918225, 0.912025, 0.9311  , 0.92315 ])

In [76]:
write_answer('q5.txt', text_cv_roc.mean())

7. Обучите логистическую регрессию на всей доступной вам выборке и выведите 5 наиболее важных для модели признаков (подумайте, какие именно признаки стоит считать такими). Вам могут пригодиться метод get_feature_names() или поле vocabulary_ у класса CountVectorizer.

In [81]:
lr_model = LogisticRegression(max_iter=1000).fit(vect_text, target)

In [88]:
lr_model.coef_

array([[ 1.05353871e-02, -1.95695878e-02, -2.48057355e-06, ...,
        -7.44834734e-03,  5.13211564e-04, -2.30642696e-03]])

In [ ]:
df_coef = pd.DataFrame()
df_coef['feature']=vectorizer.get_feature_names()
df_coef['coef']=np.abs(lr_model.coef_[0])

In [105]:
best_feat = df_coef.sort_values(by='coef', ascending=False).head(2)

In [106]:
best_feat

,feature,coef
2954,bad,0.792860
37056,unfortunately,0.628533


In [108]:
write_answer('q6.txt', *best_feat['feature'])